In [38]:
from elasticsearch import Elasticsearch
from keras.models import Model,load_model
from nltk.tokenize import sent_tokenize
import nltk
import numpy as np
from tqdm import tqdm
from keras.preprocessing import text, sequence
import pandas as pd
from keras import backend as K
import keras as ks

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
ES_settings={
    'ES_hosts':'ec2-52-34-223-218.us-west-2.compute.amazonaws.com',
'ES_user':'elastic',
'ES_password':'elasticdevpass',
    'ES_type':'plainDocs',
    'ES_index':'extractccs'
}

In [4]:
hosts=[ES_settings['ES_hosts']]
es = Elasticsearch(
       	    		hosts,
            		port=9200,
	    		http_auth=(ES_settings['ES_user'], ES_settings['ES_password']), 
	    		verify_certs=False,
            		sniff_on_start=True,    # sniff before doing anything
            		sniff_on_connection_fail=True,    # refresh nodes after a node fails to respond
            		sniffer_timeout=60, # and also every 60 seconds
            		timeout=15

)

In [5]:
search_text='self-driving'

In [15]:
response = es.search(index=ES_settings['ES_index'], body={
            "query": {
                "match": {
                    "doc": search_text
                    }
                    },
     "size" : 30

}
                   )


if response['timed_out'] == True :
                jsonresponse =[]


else:
        search_results =   response['hits']['hits']
        total_took = response['took']


In [16]:
max_features=100000
maxlen=150
embed_size=300
tok=text.Tokenizer(num_words=max_features,lower=True)


In [17]:
response

{'took': 117,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': 6531880,
  'max_score': 11.774289,
  'hits': [{'_index': 'extractccs',
    '_type': 'plainDoc',
    '_id': 'meCTCGoBuv4kx_3tVJVh',
    '_score': 11.774289,
    '_source': {'url': 'http://dailyassessment.com/2017/12/21/global-self-driving-cars-market/',
     'doc': 'The report offers a detailed overview of the global Self-driving Cars market covering key factors such as drivers and restraints impacting the growth. The global market for Self-driving Cars is segmented on the basis of manufacturers, product type, Self-driving Cars applications and regions. In addition to this, the Self-driving Cars report also forecasts market based on dominating market trends, current market conditions, and Self-driving Cars growth aspects.Initially, the report on the global Self-driving Cars market presents a complete scenario covering product/service description, future market tren

In [18]:
all_sentences=[]
urllist=[]
x=0
for X in search_results:
    #x+=1
    plaintext=X['_source']['doc'] 
    #print(plaintext)
    #print('/n')
    tok_sentence=sent_tokenize(plaintext)
    all_sentences+=tok_sentence
    num_sentence=len(tok_sentence)
    urllist+=num_sentence*[X['_source']['url'] ]
print(x)
print (len(all_sentences))
tok.fit_on_texts(all_sentences)
input_test=tok.texts_to_sequences(all_sentences)
X_test=sequence.pad_sequences(input_test, maxlen=maxlen)
y_pred = model.predict(X_test,batch_size=1024,verbose=1)
index=np.arange(len(y_pred))
y_pred=np.c_[ index,y_pred ]  
#np.append(y_pred, index, axis=1)


30
839
839/839 [==============================] - 0s 252us/step


In [ ]:
ratings = pd.DataFrame({'index':y_pred[:,0],'no_Argument':y_pred[:,1],'Argument_for':y_pred[:,2],'Argument_against':y_pred[:,3]})


In [ ]:
def compare(no_Argument,Argument_for,Argument_against):
    maxval=max(no_Argument,Argument_for,Argument_against)
    if maxval==no_Argument:
        return 'no_Argument'
    elif maxval==Argument_for:
        return 'Argument_for'
    return 'Argument_against'

In [ ]:
ratings['res']=ratings.apply(lambda x: compare(x['no_Argument'],x['Argument_for'],x['Argument_against']),axis=1)

In [ ]:
def print_argument(x):
    if x['res']!='no_Argument':
        print ("\n")
        print ("*****")
        print (x['res'])
        print (urllist[(int(x['index']))]) 
        print (all_sentences[(int(x['index']))])
    return 

In [ ]:
sortedforratings=ratings[ratings['res']=='Argument_for'].sort_values(by='Argument_for',ascending=False)
sortedAgainstRatings=ratings[ratings['res']=='Argument_against'].sort_values(by='Argument_against',ascending=False)

In [32]:
sortedforratings.apply(lambda x:print_argument(x),axis=1)

NameError: name 'sortedforratings' is not defined

In [33]:
EMB_PATH='crawl-300d-2M.vec'

In [36]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def load_embeddings(embed_dir=EMB_PATH):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in tqdm(open(embed_dir)))
    return embedding_index

def build_embedding_matrix(word_index, embeddings_index, max_features, lower = True, verbose = True):
    embedding_matrix = np.zeros((max_features, 300))
    for word, i in tqdm(word_index.items(),disable = not verbose):
        if lower:
            word = word.lower()
        if i >= max_features: continue
        try:
                embedding_vector = embeddings_index[word]
        except:
                embedding_vector = embeddings_index["unknown"]
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def build_matrix(word_index, embeddings_index):
    embedding_matrix = np.zeros((len(word_index) + 1,300))
    for word,i in  word_index.items():
        try:
            embedding_matrix[i] = embeddings_index[word]
        except:
            embedding_matrix[i] = embeddings_index["unknown"]
    return embedding_matrix

In [ ]:
embeddings_index = load_embeddings()
embedding_matrix = build_matrix(word_index, embeddings_index)

1547626it [01:54, 13532.26it/s]

In [26]:
max_features=1000000
sentenceLength=150
topic_sentenceLength=100
embed_size=300

In [28]:
word_index={'the': 1,
 'of': 2,
 'to': 3,
 'and': 4,
 'a': 5,
 'in': 6,
 'that': 7,
 'is': 8,
 'for': 9,
 'abortion': 10,
 'death': 11,
 'it': 12,
 'nuclear': 13,
 'penalty': 14}

In [ ]:
sortedAgainstRatings.apply(lambda x:print_argument(x),axis=1)

In [23]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        #self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        #self.b_constraint = constraints.get(b_constraint)

        #self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list) and len(input_shape) == 2
        print(input_shape)
        sentence_shape, topic_input_shape = input_shape
        print('sentence_shape')
        print(sentence_shape)
        print('topic_input_shape')
        print(topic_input_shape)
        #self.kernel = self.add_weight(name='kernel',
         #                             shape=(sentence_shape[1], self.output_dim),
          #                            initializer='uniform',
           #                           trainable=True)
        self.W = self.add_weight((sentence_shape[-1],topic_input_shape[-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name)),
                                 #regularizer=self.W_regularizer,
                                 #constraint=self.W_constraint)
        self.features_dim = sentence_shape[-1]

       # if self.bias:
        #    self.b = self.add_weight((input_shape[1],),
         #                            initializer='zero',
          #                           name='{}_b'.format(self.name),
           #                          regularizer=self.b_regularizer,
            #                         constraint=self.b_constraint)
       # else:
        #    self.b = None
        super(Attention, self).build(input_shape)
        #self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, arg,mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim
        x,topic_x=arg
        #eij = K.reshape(K.dot(),
        #K.reshape(self.W, (features_dim, 1))), (-1, step_dim))
            # topic_x 1*220*300 x:?220*300
        #print(self.W)
        #print('self.W')
        #print(K.reshape(self.W, (features_dim, features_dim)))
        t=K.dot(topic_x,K.reshape(self.W, (features_dim, features_dim)))
        #print(t)
        e = K.batch_dot(t, K.permute_dimensions(x, (0, 2, 1)))
        print('e')
        print(e)
        weighted_input=K.sum(e, axis=1)
        weighted_input=K.sigmoid(weighted_input)

        #tem=K.reshape(topic_x, (-1, features_dim))
        #1*220*300*300*300*300*
        #print(tem)
        #re=K.dot(tem,K.reshape(self.W, (features_dim, features_dim)))
        #print('re')
        #print(re)
        #eij=K.reshape(K.dot(re,K.reshape(x,(features_dim,-1))),(-1, step_dim,step_dim))
        #print('eij.shape')
        #print(eij.shape)
        #if self.bias:
         #   eij += self.b

        #eij = K.tanh(eij)

        #a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        #a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        weighted_input = K.expand_dims(weighted_input)
        weighted_input = x * weighted_input
        
        print('weighted_input')
        print(weighted_input)
        return weighted_input

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
    def get_config(self):
        config = {
               'W_regularizer': self.W_regularizer, 
            'W_constraint': self.W_constraint,
            'step_dim': self.step_dim,
            #'features_dim': 0
        }
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
        

In [24]:
import keras.layers as L
from keras.models import Model
from keras.optimizers import Adam

def build_model(verbose = False, compile = True):
    sequence_input = L.Input(shape=(sentenceLength,), dtype='int32')
    print(sequence_input[0])    
    topic_sequence_input = L.Input(shape=(sentenceLength,), dtype='int32')
    print(topic_sequence_input.shape)
    embedding_layer = L.Embedding(len(word_index) + 1,
                                300,
                                weights=[embedding_matrix],
                                input_length=sentenceLength,
                                trainable=False)
    print(embedding_layer)
    topic_embedding_layer=L.Embedding(len(word_index) + 1,
                                300,
                                weights=[embedding_matrix],
                                input_length=sentenceLength,
                                trainable=False)
    x = embedding_layer(sequence_input)
    print(x.shape)
    topic_x=topic_embedding_layer(topic_sequence_input)
    print(topic_x.shape)
    att = Attention(sentenceLength)([x,topic_x])
    
    #x = L.SpatialDropout1D(0.2)(x)
    print(att.shape)
    print('x.shape')
    print(x.shape)
    #att=K.Dropout(0.15)(att)
    x = L.Bidirectional(L.CuDNNLSTM(128, return_sequences=True))(att)
    print(x.shape)

    avg_pool1 = L.GlobalAveragePooling1D()(x)
    max_pool1 = L.GlobalMaxPooling1D()(x)

    x = L.concatenate([avg_pool1, max_pool1])
    print(x)

    preds = L.Dense(3, activation='sigmoid')(x)

    print(preds)
    model = Model(inputs=[sequence_input,topic_sequence_input], outputs=preds)
    if verbose:
        model.summary()
    if compile:
        model.compile(loss='binary_crossentropy',optimizer=Adam(0.005),metrics=['acc'])
    return model


In [1]:
newmodel=build_model()
newmodel.load_weights('model_weights.h5')

NameError: name 'build_model' is not defined